In [1]:
%pip install pandas
%pip install pyarrow
%pip install sleeper-py

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
from sleeperpy import *

In [5]:
base_s3_path = 's3://datalakestack-datalakebucket0256ea8e-jlou2zsfqxmr/sleeper'
past_league_ids = ['541384381865115648', '706690303247065088','784536701455429632','961113588070985728']
for index, league_id in enumerate(past_league_ids):
    year = index + 2020
    rosters = pd.DataFrame(Leagues.get_rosters(league_id))
    for week in range(1, 25):
        matchups = pd.DataFrame(Leagues.get_matchups(league_id, week))
        if not matchups.empty:
            week_result = pd.merge(rosters, matchups, on='roster_id', how='inner')
            week_result['week'] = week
            week_result['year'] = year
            if year == 2020:
                if week <= 13:
                    week_result['type'] = 'regular'
                else:
                    week_result['type'] = 'playoff'
            else:
                if week <= 14:
                    week_result['type'] = 'regular'
                else:
                    week_result['type'] = 'playoff'
            players_points_list = []
            for players_points_week in week_result['players_points'].to_list():
                players_points_list.append([{'player_id': player_id, 'points': points} for player_id, points in players_points_week.items()])
            week_result['players_points'] = players_points_list
            week_result = week_result.drop(columns=['settings', 'metadata'])
            week_result.to_parquet(f'{base_s3_path}/matchups/{year}/{week}/result.parquet', engine='pyarrow')

In [6]:
matchups = pd.read_parquet('s3://datalakestack-datalakebucket0256ea8e-jlou2zsfqxmr/sleeper/matchups/')

In [7]:
print(matchups.columns)

Index(['co_owners', 'keepers', 'league_id', 'owner_id', 'player_map',
       'players_x', 'reserve', 'roster_id', 'starters_x', 'taxi', 'points',
       'players_y', 'custom_points', 'matchup_id', 'starters_y',
       'starters_points', 'players_points', 'week', 'year', 'type'],
      dtype='object')
